# Script for 2025 Nextflow Microbiome Workshop

## Sol login and Fastq file upload

In [ ]:
# Make sure you copy the files to your home directory on sol. You can access the interactive Sol browser at with the link below:
# Sol --> https://ood05.sol.rc.asu.edu/pun/sys/dashboard/ 

# You can also use scp or rsync 
# Here is an example: 
# rsync -P *.fastq (ASURITE USERNAME)@login.sol.rc.asu.edu:/home/[ASURITE USERNAME]

# it is recommended you create a folder for storing all of the pipelines results. You can create directories by running the following command while logged into Sol:
# mkdir nextflow_pacbio_2025_workshop

# login into sol with the following: 
# ssh (ASURITE USERNAME)@login.sol.rc.asu.edu

# if having difficulties connecting, or need to connect to sol while off campus, please see the ASU Supercomputing Wiki:
# Connecting to Sol --> https://asurc.atlassian.net/wiki/spaces/RC/pages/1905131521/Connecting+to+the+Supercomputers+with+SSH
# Transferrring Files to a Supercomputer --> https://asurc.atlassian.net/wiki/spaces/RC/pages/1852670115/Transferring+Files+to+a+Supercomputer 

In [ ]:
# In the workshop github, there are two important files provided if running the nextflow pipeline included at the end of the workshop script: 
# workshop_metadata.tsv and workshop_samplelist.tsv

# Before running the pipeline, open the workshop_samplelist.tsv document and ensure that the filepaths for the fastq files map
# to where they are on your Sol account. Nextflow is picky about the column names, and extra "lines" in the samplelist
# will cause the pipeline to fail at startup. Good first place to check for debugging! 

## FastQC

In [ ]:
# We will first assess the quality of the data using fastqc ls
# Fastqc is installed on Sol, and publically available through modules
# To access it, we can use the code below
module load fastqc-0.12.1-gcc-11.2.0

# Navigate to the directory containing your fastq files
# To run fastqc on one sample: 
fastqc mouse_1.fastq

# To run fastqc on all fastq files in a directory 
fastqc *.fastq

## Seqkit

In [ ]:
# Use seqkit to gain more insights about the quality of the data 

# Use the following to install seqkit onto nextflow environment
conda install bioconda::seqkit

# Use seqkit stats 
seqkit stats -a *.fastq > seqkit-stats.txt # open in excel 

## Create QIIME2 environment

In [ ]:
# We will need to create a conda environment for the new QIIME2 environment

wget https://data.qiime2.org/distro/amplicon/qiime2-amplicon-2024.5-py39-linux-conda.yml

# You may need to start an interactive session for this to work
# example: interactive -c 4 --mem=48G
conda env create -n qiime2-amplicon-2024.5 --file https://data.qiime2.org/distro/amplicon/qiime2-amplicon-2024.5-py39-linux-conda.yml

# Activate the environment
source activate activate qiime2-amplicon-2024.5

In [ ]:
# Because the following steps will need more RAM, we need to submit slurm batch scripts 

# First, update all filepaths in the below code to your workshop files stored on Sol
# Then, copy the entire code below into a text file and save as run_nf.slurm

# Submit the job
sbatch run_nf.slurm

# Check your job in the queue 
squeue -u ASURITEUSER

# To cancel the job if something is wrong (replace 123456 with your job ID)
scancel 123456

## Import data into QIIME2

In [ ]:
#!/bin/bash

#SBATCH --job-name=one-import_fastq_files
#SBATCH -o import_fastq.out
#SBATCH --nodes=1
#SBATCH -t 8:00:00
#SBATCH --cpus-per-task=10
#SBATCH --mem=40G
#SBATCH -p general

cd $SLURM_SUBMIT_DIR

set -uex

# Activate QIIME2 
module load mamba/latest
source activate qiime2-amplicon-2024.5

# Import FASTQ FILES
# You will need to make sure you include the manifest file for these samples
qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path manifest.txt \
  --input-format SingleEndFastqManifestPhred33V2 \
  --output-path pacbio-seqs.qza

# Denoising the sequencing data 

Top code cell demonstrates DADA2 denoising for raw PacBio sequences

The bottom code cell is what we will use today, since the primers for the PacBio sequence samples we are working with have already been trimmed. 

In [ ]:
#!/bin/bash
#SBATCH --job-name=denoise
#SBATCH -o denoise.out
#SBATCH --nodes=1
#SBATCH -t 8:00:00
#SBATCH --cpus-per-task=10
#SBATCH --mem=40G
#SBATCH -p general

cd $SLURM_SUBMIT_DIR

set -uex

# Record the start time
start_time=$(date)

module load mamba/latest

source activate QIIME2-amplicon-2024.5

start_time=$(date)

qiime dada2 denoise-ccs \
--i-demultiplexed-seqs pacbio-seqs.qza \
--p-trim-left 0 \
--p-trunc-len 0 \
--p-front AGRGTTYGATYMTGGCTCAG \
--p-adapter RGYTACCTTGTTACGACTT \
--o-representative-sequences rep-seqs.qza   \
--o-table table.qza   \
--o-denoising-stats stats.qza

end_time=$(date +%s)

runtime=$((end_time - start_time))

echo "Job complete"


In [ ]:
#!/bin/bash
#SBATCH --job-name=denoise
#SBATCH -o denoise.out
#SBATCH --nodes=1
#SBATCH -t 8:00:00
#SBATCH --cpus-per-task=10
#SBATCH --mem=40G
#SBATCH -p general

cd $SLURM_SUBMIT_DIR

set -uex

# Record the start time
start_time=$(date)

module load mamba/latest

source activate QIIME2-amplicon-2024.5

start_time=$(date)

qiime dada2 denoise-single \
  --i-demultiplexed-seqs pacbio-seqs.qza \
  --p-trunc-len 0 \
  --p-trim-left 0 \
  --o-representative-sequences rep-seqs.qza \
  --o-table table.qza \
  --o-denoising-stats stats.qza

end_time=$(date +%s)

runtime=$((end_time - start_time))

echo "Job complete"

## Download Greengenes2 database

In [ ]:
# Download the greengenes2 database 
wget --no-check-certificate https://ftp.microbio.me/greengenes_release/2024.09/2024.09.backbone.full-length.fna.qza

# For the taxonomy file
wget --no-check-certificate https://ftp.microbio.me/greengenes_release/2024.09/2024.09.backbone.tax.qza

## Classify with Vsearch

In [ ]:
#!/bin/bash

#SBATCH --job-name=classify
#SBATCH -o classify.out
#SBATCH --nodes=1
#SBATCH -t 4:00:00
#SBATCH --cpus-per-task=10
#SBATCH --mem=40G
#SBATCH -p general

cd $SLURM_SUBMIT_DIR

set -uex

# Record the start time
start_time=$(date)

module load mamba/latest

source activate QIIME2-amplicon-2024.5

qiime feature-classifier classify-consensus-vsearch \
    --i-query rep-seqs.qza \
    --i-reference-reads 2024.09.backbone.full-length.fna.qza  \
    --i-reference-taxonomy 2024.09.backbone.tax.qza  \
    --p-maxaccepts 1 --p-strand "plus" \
    --p-threads 4 \
    --verbose \
    --output-dir taxa

qiime tools export --input-path taxa/classification.qza --output-path taxa

echo "Job complete"


### Analyses 

In [ ]:
# convert taxonomy.tsv file to taxonomy.qza file
qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path taxonomy.tsv \
  --output-path taxonomy.qza

# change the header from Feature ID to feature-id
# run the following to check the file
# qiime metadata tabulate --m-input-file taxonomy.qza --o-visualization taxonomy.qzv

In [ ]:
# Create a barplot

qiime taxa barplot \
--i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--m-metadata-file workshop_metadata.tsv \
--o-visualization taxa-bar-plots.qzv

# used qiime taxa barplot --i-table table.qza --i-taxonomy taxa/classification.qza --m-metadata-file workshop_metadata.tsv --o-visualization taxa-bar-plots.qzv


In [ ]:
#!/bin/bash

#SBATCH --job-name=denoise
#SBATCH -o denoise.out
#SBATCH --nodes=1
#SBATCH -t 4:00:00
#SBATCH --cpus-per-task=10

cd $SLURM_SUBMIT_DIR

set -uex

# collapse table to the genus level 
qiime taxa collapse --i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--p-level 6 \
--o-collapsed-table genus-table.qza

qiime taxa collapse --i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--p-level 6 \
--o-collapsed-table genus-table-2.qza

qiime taxa collapse --i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--p-level 7 \
--o-collapsed-table species-table.qza

qiime taxa collapse --i-table table.qza \
--i-taxonomy taxa/taxonomy.qza \
--p-level 2 \
--o-collapsed-table phylum-table.qza

echo "Job complete"

In [ ]:
# Convert the table from a QZA format to a BIOM or txt format 
# Note that both of these files can be inputted into other programs 

qiime tools export --input-path genus-table.qza --output-path genus-analysis-2
biom convert -i genus-analysis/feature-table.biom -o genus-analysis/feature-table.txt --to-tsv

qiime tools export --input-path species-table.qza --output-path species-analysis
biom convert -i species-analysis/feature-table.biom -o species-analysis/feature-table.txt --to-tsv

In [ ]:
# What if you want to remove singletons and rare taxa that may be inflating diversity estimates and are present due to spurious mappings 

qiime feature-table filter-features \
  --i-table genus-table-2.qza \
  --p-min-samples 2 \
  --p-min-frequency 10 \
  --o-filtered-table filtered-genus-table.qza


## Compositional analyses

In [ ]:
# install gemelli
# make sure your QIIME2 environment is active 
pip install gemelli

In [ ]:
#create a Aitchison distance matrix 

qiime gemelli rpca \
--i-table genus-table.qza \
--o-biplot genus-table-ordination.qza \
--o-distance-matrix genus-table-distance.qza

qiime emperor biplot \
--i-biplot genus-table-ordination.qza \
--m-sample-metadata-file metadata.txt \
--o-visualization genus-biplot.qzv \
--p-number-of-features 1
    
# PERMANOVA results 

NAME=genus
METADATA=metadata.txt
declare -a StringArray=("GroupNumber" "Health")
for category in ${StringArray[@]}; do
qiime diversity beta-group-significance \
  --i-distance-matrix genus-table-distance.qza \
  --m-metadata-file $METADATA \
  --m-metadata-column "$category" \
  --o-visualization ${NAME}-$category-significance.qzv \
  --p-pairwise
done

qiime diversity adonis \
--i-distance-matrix genus-table-distance.qza \
--m-metadata-file $METADATA \
--p-formula "GroupNumber*Health" \
--o-visualization adonis-${NAME}.qzv

## Alpha diversity 

In [ ]:
METRIC=observed_features
qiime diversity alpha \
--i-table genus-table.qza \
--p-metric $METRIC \
--o-alpha-diversity alpha-${NAME}-${METRIC}.qza

qiime diversity alpha-group-significance
--i-alpha-diversity alpha-${NAME}-${METRIC}.qza
--m-metadata-file $METADATA
--o-visualization alpha-${NAME}-${METRIC}.qzv

Additional commands

In [ ]:
# obc2dfatq command

obc2fastq --input <run folder> \
--output <output folder name> \
--flowcellid <flow cell ID> \
--samplesheet <sampleSheet.csv file name> \
--designsheet <obc2fastq_params file name> \
--threadlanes <int> \--threadsperlane <int> \
--controlsfile <control fasta file name> \
--barcodeallowedmismatches <int> \

In [ ]:
# Subsample the fastq files to make them smaller 
# Good when you want to just test whether your program is working without having to do it on the entire dataset

#!/bin/bash

#SBATCH --job-name=denoise
#SBATCH -o denoise.out
#SBATCH --nodes=1
#SBATCH -t 4:00:00
#SBATCH --cpus-per-task=10

cd $SLURM_SUBMIT_DIR

set -uex

# Record the start time
start_time=$(date)

module load mamba/latest

source activate FASTQ_PROCESSING

ls SRR2338099*.fastq | while read samples; do
    base_name=$(basename "$samples" .fastq)  # Extract the base name without .fastq
    seqkit sample -p 0.4 "$samples" -o "${base_name}.4.fastq"  # Use the base name for output
done

# -p is for proportion of sequences to subsample to; here we are collecting 40% of sequences

echo "Job Complete"


In [ ]:
# If you are having trouble with DEICODE, you may need to downgrade your numpy environment
# Make sure your conda environment for QIIME2 is active conda 
mamba install numpy=1.19.5

## Create Nextflow conda environment

In [ ]:
# We will need to create a conda environment for the nextflow pipeline environment

# First, request an interactive environment while logged into sol (navigate to "System" then "Sol Shell Access" if on Sol web browser)  
interactive -c 4 --mem=48G

# We then load the mamba module, and create a new conda environment for nextflow. 
module load mamba/latest
conda create -n nextflow -c bioconda -c conda-forge nextflow

# Activate the new environment
source activate nextflow

# Confirm Nextflow was corretly installed
nextflow info

# To update Nextflow, run:
nextflow self-update

# After installing Nextflow, we have to download the alignment databases using the following commands. 
# To update the pipeline in the future, type "git pull" instead of "git clone".
# This will take awhile to download, please download these prior to the workshop. (Took around ~35 mins when testing)
git clone https://github.com/PacificBiosciences/HiFi-16S-workflow.git
cd HiFi-16S-workflow
nextflow run main.nf --download_db

## Running the Nextflow Pipeline

In [ ]:
#!/bin/bash
#SBATCH -J nextflow-pb16s
#SBATCH -p general
#SBATCH -N 1
#SBATCH -c 32
#SBATCH --mem=128G
#SBATCH -t 8:00:00
#SBATCH -o logs/%x_%j.out
#SBATCH -e logs/%x_%j.err
#SBATCH --export=NONE

# change directory where we submitted the slurm job
cd "$SLURM_SUBMIT_DIR"

set -uex

# Record start time
start_time=$(date)

# Load Modules 
module load mamba/latest

# Activate nextflow environment
source activate nextflow

# Metadeta and Sample list inputs. Ensure these match where you uploaded your files on sol
SAMPLELIST="/home/**ASURITE USERNAME**/nextflow_pacbio_2025_workshop/workshop_samplelist.tsv"
METADATA="/home/**ASURITE USERNAME**/nextflow_pacbio_2025_workshop/workshop_metadata.tsv"

# Set the directory where you want your nextflow results to go. 
RESULTS_DIR="/home/**ASURITE USERNAME**/nextflow_pacbio_2025_workshop/results"

# Automatically makes a "error logs" folder for to store nextflow run and error logs. 
mkdir -p "$RESULTS_DIR" "logs"

# Run a basic Nextflow workflow
nextflow run /home/**ASURITE USERNAME**/HiFi-16S-workflow/main.nf \
  --input "$SAMPLELIST" \
  --metadata "$METADATA" \
  --min_len 900 \
  --max_len 1700 \
  --dada2_cpu 32 \
  --vsearch_cpu 32 \
  -resume

# there are other possible inclusions for the pipeline, such as:
# --run_picrust2 \        --> runs a picrust2 analysis in the pipeline. 
# --rarefaction_depth \   --> Rarefaction curve "max-depth" parameter. (automatically determined by default)
# --front_p    Forward primer sequence.
# --adapter_p    Reverse primer sequence.

# for the full list of nextflow options, type:
# nextflow run main.nf --help           

# Record end time   
end_time=$(date)

# Indicate the pipeline has finished the analysis. 
echo "Start Time: $start_time"
echo "End Time: $end_time"
echo "Job Complete"

## Understanding Your PacBio HiFi 16S Nextflow Outputs

When the HiFi-16S Nextflow pipeline finishes, you’ll have a directory of outputs (we called it `RESULTS_DIR` in the Slurm script).  
Here’s a practical guide to what the main files/folders contain and how you might use them.

## PacBio HiFi 16S Nextflow Pipeline Outputs

### Feature Table & Sample Data  
**Directory:** `tables/`

- **feature_table.qza** or **table.qza**  
  QIIME 2 feature table (ASV table).  
  Rows = ASVs, columns = samples, values = counts.

- **rarefied_table.qza**  
  Feature table rarefied to an even sequencing depth.  
  Used for alpha and beta diversity analyses.


### Representative Sequences (ASVs)  
**Directory:** `dada2/` or `tables/`

- **rep_seqs.qza** or **rep_seqs_denoised.qza**  
  Representative ASV sequences in FASTA format (one sequence per feature ID).  
  Used for taxonomy assignment, phylogenetic tree building, and downstream sequence analyses.

### Taxonomy Assignments  
**Directory:** `taxonomy/`

- **taxonomy.qza**  
  QIIME 2 artifact mapping ASV IDs to taxonomic classifications.

- **taxonomy.qzv**  
  QIIME 2 visualization of taxonomic composition (bar plots, metadata grouping, etc.).


### Phylogenetic Tree  
**Directory:** `phylogeny/`

- **aligned_rep_seqs.qza**  
  Multiple sequence alignment of representative ASVs (e.g., MAFFT).

- **phylogeny_unrooted.qza**  
  Unrooted phylogenetic tree built from aligned ASVs.

- **phylotree_mafft_rooted.qza** or **phylogeny_rooted.qza**  
  Rooted phylogenetic tree required for phylogeny-based metrics (Faith’s PD, Weighted/Unweighted UniFrac).


### Quality Control & Read Statistics  
**Directory:** `qc/`

- Per-sample QC summaries (HTML/TSV/PNG).  
  Includes read counts, filtering results, read length distributions, and error profiles.


### Pipeline & Run Logs  
**Directories:** `logs/` and/or `nextflow_reports/`

- **nextflow.log** or **.nextflow.log**  
  Main pipeline execution log.

- **execution_report.html**  
  Summary of pipeline tasks, resource usage, and performance.

- **execution_timeline.html**  
  Gantt-style timeline of process execution.

- **execution_trace.txt**  
  Tabular summary of CPU/memory/time usage for each process.


### Functional Profiling (Optional, if `--run_picrust2` used)  
**Directory:** `picrust2/`

- KEGG Ortholog abundance tables  
- Predicted pathway abundance tables  
  (Outputs from PICRUSt2 functional prediction.)


### Files Used in QIIME 2 Post-Analysis  
**Used from Nextflow output or external files**

- **rarefied_table.qza**  
- **phylotree_mafft_rooted.qza**  
- **sample_metadata.tsv** (external)  
  Inputs for any downstream analysis including alpha/beta diversity, PERMANOVA, and visualization steps.


## For differential abundance analysis
I recommend Maaslin2 or Maaslin3. Unlike other differential abundance analysis, this tool has the capability of log transforming abudnances while also accounting for both fixed and random effects 

link: https://huttenhower.sph.harvard.edu/maaslin/
